In [1]:
import json
import re
from math import log10

import pandas as pd

In [2]:
# reduce $MESA_DIR/chem/data/isotopes.data

build_isotopes = False

if build_isotopes:
    with open('isotopes.data', 'r') as f:
        lines = f.readlines()

    isotopes = {}

    for line in lines:
        if line == '\n': continue
        segments = line.split()
        iso = segments[0]

        if iso.startswith('xtra'): continue
        if iso[0].isalpha() and iso[-1].isdigit():
            isotopes[iso] = float(segments[1])

    with open('isotopes.json', 'w') as f:
        json.dump(isotopes, f)
    del isotopes

In [3]:
with open('isotopes.json', 'r') as f:
    isotopes = json.load(f)

# build list of elements

elements = []

for iso in isotopes:
    m = re.match('[a-z]+', iso)
    elem = m.group(0).capitalize()

    if not elements or elem != elements[-1]:
        elements.append(elem)

In [4]:
fractions = ['X_frac', 'Y_frac', 'Z_frac', 'X/Z', 'OP/Z']
df = pd.DataFrame(columns=[], index=fractions + elements)

In [5]:
def clean_l(line: str) -> str:
    return re.sub('[$\\\[\]()\n<]', '', line)

def clean_s(seg: str) -> str:
    if 'pm' in seg:
        seg = seg.partition('pm')[0]
    return seg.strip()

In [6]:
volatile_elements = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn']

def is_volatile(elem: str) -> bool:
    assert elem in elements, 'No such element'
    return elem in volatile_elements

In [7]:
def met_scale(ref: str) -> None:
    df[f'{ref}S'] = -999.99
    delta = df.at['Si', f'{ref}P'] - df.at['Si', f'{ref}M']

    for elem in elements:
        phot = df.at[elem, f'{ref}P']
        met  = df.at[elem, f'{ref}M']
        if max(phot, met) == -999.99: continue

        if is_volatile(elem):
            df.at[elem, f'{ref}S'] = df.at[elem, f'{ref}P']
        else:
            df.at[elem, f'{ref}S'] = df.at[elem, f'{ref}M'] + delta

        if ref in ['G98', 'M22'] and elem in ['Kr', 'Xe']:
            df.at[elem, f'{ref}S'] = df.at[elem, f'{ref}M'] + delta

In [8]:
# parse segments of chem/public/chem_def.f90

p_mesa = re.compile('zfrac\(e_([A-Za-z]+)\s*\)\s*=\s*([\-0-9\.]+)d0')

def parse_chem_def(col: str, filename: str) -> None:
    df[col] = -20.0

    with open(filename) as f:
        lines = f.readlines()

    for line in lines:
        m = re.search(p_mesa, line.strip())
        if m is None: continue
        elem, mesa = m.group(1).capitalize(), float(m.group(2))
        df.at[elem, col] = mesa

In [9]:
# parse GS98.txt

df['G98P'] = -999.99
df['G98M'] = -999.99

with open('GS98.txt') as f:
    lines = f.readlines()

for line in lines:
    segments = [clean_s(seg) for seg in clean_l(line).split('&')]

    for i in range(len(segments) // 4):
        elem, phot, met, delta = segments[i*4:(i+1)*4]
        if not elem: continue
        z, elem = elem.partition(' ')[::2]

        # if int(z) != elements.index(elem)+1:
        #     print('Warning: z and elem do not match for', elem)

        # if phot and met and delta:
        #     if round(float(phot) - float(met) - float(delta), 2) != 0.0:
        #         print('Warning: phot, met, and delta do not match for', elem)

        if phot: df.at[elem, 'G98P'] = float(phot)
        if met:  df.at[elem, 'G98M'] = float(met)

met_scale('G98')
parse_chem_def('G98D', 'GS98_mesa.txt')  # d stands for MESA default

In [10]:
# parse MBS22.txt

df['M22P'] = df['G98P']
df['M22M'] = df['G98M']

with open('MBS22.txt') as f:
    lines = f.readlines()

for line in lines:
    if line.startswith('%'): continue

    segments = [clean_s(seg) for seg in clean_l(line).split('&')]
    elem, phot, note, met = segments

    if phot != '-': df.at[elem, 'M22P'] = float(phot)
    if met != '-':  df.at[elem, 'M22M'] = float(met)

for elem, abun in zip(['Ba', 'Eu', 'Hf', 'Os', 'Th'],
                      [2.27, 0.52, 0.73, 1.36, 0.08]):
    df.at[elem, 'M22P'] = abun

met_scale('M22')

In [11]:
# parse AAG21.txt

df['A21P'] = -999.99
df['A21M'] = -999.99

with open('AAG21.txt') as f:
    lines = f.readlines()

for line in lines:
    segments = [clean_s(seg) for seg in clean_l(line).split('&')]

    for i in range(2):
        z, elem, phot, met, note = segments[i*5:(i+1)*5]
        if not z: continue
        if phot: df.at[elem, 'A21P'] = float(phot)
        if met:  df.at[elem, 'A21M'] = float(met)

met_scale('A21')

In [12]:
# parse AGSS09.txt

df['A09P'] = -999.99
df['A09M'] = -999.99

with open('AGSS09.txt') as f:
    lines = f.readlines()

for line in lines:
    segments = [clean_s(seg) for seg in clean_l(line).split('&')]

    for i in range(len(segments) // 4):
        z, elem, phot, met = segments[i*4:(i+1)*4]
        if phot: df.at[elem, 'A09P'] = float(phot)
        if met:  df.at[elem, 'A09M'] = float(met)

met_scale('A09')
parse_chem_def('A09D', 'AGSS09_mesa.txt')  # d stands for MESA default

In [13]:
def get_weight(wt: str) -> float:
    return float(re.sub('[$\[\]() ]', '', wt))

In [14]:
# parse IUPAC 2021 Atomic Weights

build_weights = False

if build_weights:
    with open('IUPAC21.txt') as f:
        lines = f.readlines()

    weights = {}

    for line in lines:
        segments = line.rstrip().split('\t')
        elem, wt = segments[1::2]
        weights[elem] = get_weight(wt)

In [15]:
# parse $MESA_DIR/chem/data/lodders03.data

if build_weights:
    with open('lodders03.data') as f:
        lines = f.readlines()

    weights_l = {}  # l stands for lodders

    for line in lines:
        if line.startswith('!'): continue
        segments = line.split()
        z, elem, a, frac, n = segments

        if elem not in weights_l:
            weights_l[elem] = isotopes[f'{elem.lower()}{a}'] * float(frac)/100.0
        else:
            weights_l[elem] += isotopes[f'{elem.lower()}{a}'] * float(frac)/100.0

In [16]:
if build_weights:
    # for elem in weights_l:
    #     print(elem, log10(abs(weights_l[elem] / weights[elem] - 1)))

    weights.update(weights_l)

    with open('weights.json', 'w') as f:
        json.dump(weights, f)
    del weights

In [17]:
with open('weights.json', 'r') as f:
    weights = json.load(f)

In [18]:
OP_elements_main = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'S', 'Ca', 'Ar', 'Cr', 'Mn', 'Fe', 'Ni']
OP_elements_plus = ['P', 'Cl', 'K', 'Ti']
OP_elements = OP_elements_main + OP_elements_plus

def is_in_OP(elem: str) -> bool:
    assert elem in elements, 'No such element'
    return elem in OP_elements

In [19]:
def calc_mass_fractions(col: str) -> None:
    X = 10.0 ** df.at['H',  col] * weights['H' ]
    Y = 10.0 ** df.at['He', col] * weights['He']

    Z, OP = 0.0, 0.0
    for metal in elements[2:]:
        fraction = 10.0 ** df.at[metal, col] * weights[metal]
        Z += fraction
        if is_in_OP(metal): OP += fraction
    total = X + Y + Z

    df.at['X_frac', col] = X / total
    df.at['Y_frac', col] = Y / total
    df.at['Z_frac', col] = Z / total
    df.at['Z/X',    col] = (Z / X) if X else 0.0
    df.at['OP/Z',   col] = OP / Z

In [20]:
for col in df.columns:
    calc_mass_fractions(col)

In [21]:
# print(df.to_markdown())
df.to_excel('solmix.xlsx')